# Análisis de TRayectorias de GPS de Vehículos con RDDs

In [35]:
from pyspark.sql import SparkSession
from datetime import datetime
from math import radians, sin, cos, sqrt, atan2
import os

In [2]:
spark = SparkSession.builder.appName("GPS Analysis RDD").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/17 12:06:53 WARN Utils: Your hostname, DESKTOP-UEO6NL9 resolves to a loopback address: 127.0.1.1; using 172.31.181.188 instead (on interface eth0)
25/04/17 12:06:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/17 12:06:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
rdd = spark.sparkContext.textFile("../data/input/vehicle_cdmx.csv")
header = rdd.first()
rdd = rdd.filter(lambda x: x != header)
rdd.take(10)

['vehicle543,2024-01-15T23:23:00,19.258204,-99.073388',
 'vehicle856,2024-01-19T17:38:00,19.345764,-99.076983',
 'vehicle697,2024-01-03T02:24:00,19.269523,-99.040627',
 'vehicle996,2024-01-09T19:15:00,19.446935,-99.265204',
 'vehicle102,2024-01-27T10:09:00,19.319206,-99.035212',
 'vehicle282,2024-01-02T14:11:00,19.261094,-99.261406',
 'vehicle861,2024-01-14T08:47:00,19.349614,-99.143884',
 'vehicle913,2024-01-10T10:36:00,19.541614,-99.147837',
 'vehicle765,2024-01-21T08:42:00,19.539086,-99.148234',
 'vehicle222,2024-01-18T16:38:00,19.330292,-99.216332']

In [7]:
# parsear y agrupar por vehículo
def parse_line(line):
    fields = line.split(",")
    return (
        fields[0],
        (datetime.fromisoformat(fields[1]), float(fields[2]), float(fields[3])),
    )

In [8]:
vehicle_rdd = rdd.map(parse_line)
grouped_rdd = vehicle_rdd.groupByKey().mapValues(lambda v: sorted(v))
grouped_rdd.take(1)

[('vehicle543',
  [(datetime.datetime(2024, 1, 1, 0, 12), 19.477575, -99.226012),
   (datetime.datetime(2024, 1, 1, 1, 33), 19.506133, -99.095407),
   (datetime.datetime(2024, 1, 1, 1, 52), 19.538573, -99.126471),
   (datetime.datetime(2024, 1, 1, 2, 36), 19.38209, -99.013092),
   (datetime.datetime(2024, 1, 1, 2, 54), 19.398089, -99.252483),
   (datetime.datetime(2024, 1, 1, 4, 35), 19.252832, -99.112012),
   (datetime.datetime(2024, 1, 1, 5, 45), 19.471339, -99.145895),
   (datetime.datetime(2024, 1, 1, 5, 50), 19.507803, -99.039505),
   (datetime.datetime(2024, 1, 1, 6, 32), 19.393046, -99.299478),
   (datetime.datetime(2024, 1, 1, 7, 7), 19.330842, -99.131915),
   (datetime.datetime(2024, 1, 1, 8, 26), 19.468059, -99.297911),
   (datetime.datetime(2024, 1, 1, 11, 13), 19.361279, -99.284367),
   (datetime.datetime(2024, 1, 1, 11, 20), 19.320672, -99.062833),
   (datetime.datetime(2024, 1, 1, 12, 14), 19.291773, -99.218942),
   (datetime.datetime(2024, 1, 1, 12, 30), 19.352527, -99.2

In [9]:
# calcular la distancia entre puntos consecutivos
def haversine_formula(p1, p2):
    R = 6371
    lat1, lon1 = radians(p1[1]), radians(p1[2])
    lat2, lon2 = radians(p2[1]), radians(p2[2])
    dlat = lat2 - lat1
    dlon = lon1 - lon2
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [ ]:
def compute_distance(rdd):
    total = 0
    for i in range(1, len(rdd)):
        total += haversine_formula(rdd[i - 1], rdd[i])
    return total

In [12]:
distance_rdd = grouped_rdd.mapValues(compute_distance)
distance_rdd.take(5)

[('vehicle543', 16895.23650169153),
 ('vehicle697', 16883.891625771907),
 ('vehicle861', 17883.373715783237),
 ('vehicle913', 16146.48009986393),
 ('vehicle765', 16429.596216678656)]

In [30]:
# detectar trayectorias sospechosas (demasiado largas)
anomalies_rdd = distance_rdd.filter(lambda x: x[1] > 18000)
anomalies_rdd.take(5)

[('vehicle013', 18174.079387985224),
 ('vehicle294', 18288.444549428634),
 ('vehicle225', 18197.514768267003),
 ('vehicle656', 18461.45916940951),
 ('vehicle531', 18068.583563975324)]

In [31]:
# crear un mapa de calor por coordenadas
def grid_cell(lat, lon, precisio=3):
    return (round(lat, precisio), round(lon, precisio))

In [ ]:
grid_cells = vehicle_rdd.map(lambda x: grid_cell(x[1][1], x[1][2]))
heat_map = grid_cells.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

In [34]:
distance_rdd.saveAsTextFile("../data/output/distancias")
anomalies_rdd.saveAsTextFile("../data/output/anomalias")
heat_map.saveAsTextFile("../data/output/mapa_calor")

In [37]:
with open("mapacalor.txt", "w") as outfile:
    for fname in os.listdir("../data/output/mapa_calor/"):
        if fname.startswith("part-"):
            with open(os.path.join("../data/output/mapa_calor/", fname)) as infile:
                outfile.write(infile.read())